<center> 
    <h1>HW4 - Scaling Laws </h1>
</center>
<h5>Student: Ian Marchetti Meirelles</h5>

# Scaling Laws

In this week we want to create simple scaling laws for language models. We will follow approach two in Hoffmann et al. 2022 https://arxiv.org/abs/2203.15556.

- We train GPT models based on nanoGPT: https://github.com/karpathy/nanoGPT

- Download the dataset gutenberg_poetry.txt from MOODLE. This text is a filtered version of https://huggingface.co/datasets/biglam/gutenberg-poetry-corpus.

- You can do the training in https://colab.google/.



In [2]:
# Scaling laws will be based on the following models.
gpt_models = {
    'gpt-0':    dict(n_layer=2, n_head=2, n_embd=96),
    'gpt-1':    dict(n_layer=4, n_head=4, n_embd=96),
    'gpt-2':    dict(n_layer=9, n_head=8, n_embd=96),
    'gpt-3':    dict(n_layer=10, n_head=8, n_embd=128),
    'gpt-4':    dict(n_layer=20, n_head=16, n_embd=128),
}

Task 1: Encode the gutenberg_poetry.txt file using prepare.py from nanoGPT/shakespeare_char. Create the models from gpt_models (see above) and calculate the number of non-embedding paramters $N$. (Hint: see model.py)


Note: All code modifications made to the nanoGPT repository are included in this notebook for reference. However, the model was actually trained using the corresponding .py files, not within the notebook itself."
Repository fork url: https://github.com/ianmm2d/nanoGPT-foundation-models

Encoding, based on shakespeare_char/prepare.py:

In [ ]:
"""
Prepare the Gutenberg Poetry dataset for character-level language modeling.
So instead of encoding with GPT-2 BPE tokens, we just map characters to ints.
Will save train.bin, val.bin containing the ids, and meta.pkl containing the
encoder and decoder and some other related info.
"""
import os
import pickle
import numpy as np

# This is just the correct code that was saved in prepare.py for the activity 
# it was not created to run inside jupyter notebook
# For that, it is necessary to change the relative file path

input_file_path = os.path.join(os.path.dirname(__file__), 'gutenberg_poetry.txt')

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(os.path.dirname(__file__), 'train.bin'))
val_ids.tofile(os.path.join(os.path.dirname(__file__), 'val.bin'))

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(os.path.join(os.path.dirname(__file__), 'meta.pkl'), 'wb') as f:
    pickle.dump(meta, f)


Creating the GPT-Models using model.py from nanoGPT for params calculation:
* Using vocab_size as the vocab_size obtained from encoding;
* Defining block_size as 32, required for task 2.

In [3]:
"""
Full definition of a GPT Language Model, all of it in this single file.
References:
1) the official GPT-2 TensorFlow implementation released by OpenAI:
https://github.com/openai/gpt-2/blob/master/src/model.py
2) huggingface/transformers PyTorch implementation:
https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py
"""

import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
        for block in self.transformer.h:
            if hasattr(block.attn, 'bias'):
                block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        override_args = override_args or {} # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == 'dropout' for k in override_args)
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        config_args['bias'] = True # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if 'dropout' in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args['dropout'] = override_args['dropout']
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [13]:
import pickle

with open('data/gutenberg/meta.pkl', 'rb') as f:
    meta = pickle.load(f)

GPT_configs = {}
GPT_model = {}

for gpt in gpt_models:
    GPT_configs[gpt] = GPTConfig(
        vocab_size=meta['vocab_size'],
        block_size=32,
        **gpt_models[gpt]
    )
    GPT_model[gpt] = GPT(GPT_configs[gpt])

number of parameters: 0.23M
number of parameters: 0.46M
number of parameters: 1.02M
number of parameters: 2.00M
number of parameters: 3.98M


Calculating the number of Non-Embedding parameters using the built-in method from GPT class.

In [14]:
for model in GPT_model:
    print(f'{model}: {GPT_model[model].get_num_params():}')

gpt-0: 232992
gpt-1: 456672
gpt-2: 1015872
gpt-3: 1995136
gpt-4: 3977856


Number of parameters:
| model         | num_params   |
|---------------|--------------|
| GPT-0         | 232992       |
| GPT-1         | 456672       |
| GPT-2         | 1015872      |
| GPT-3         | 1995136      |
| GPT-4         | 3977856      |

Task 2: For the models from above we vary the number of training tokens such that we have constant amount of compute $C$. The compute budget can be approximated by $C \approx 6ND$ where $N$ is the model size and $D$ the number of processed training tokens. We fix three compute values $C_0=6\cdot 10^{13}$, $C_1=3\cdot 10^{14}$, and $C_2=6\cdot 10^{14}$. For $C_0$, we train the models gpt-0, gpt-1, gpt_2, and gpt-3. For $C_1$, we use gpt-1, gpt-2, and gpt-3. For $C_2$, use the models gpt-1, gpt-2, gpt-3, and gpt-4. Derive a formula for the number of training steps $S$ and create a table with the corresponding training steps for each training run. We assume fixed context length of $32$ and batch size of $64$. We train on one GPU with gradient accumulation steps of 1.  


Deriving the formula for the number of training steps S:

Let \( D \) denote the total number of tokens processed during training. It can be expressed as the product of the batch size \( B \), block size (or context length) \( T \), and the total number of training steps \( S \):

$$ D = B \times T \times S $$

The total computational cost \( C \) of training a transformer model can be approximated by:

$$ C = 6 \cdot N \cdot D $$

where \( N \) represents the number of non-embedding parameters in the model.

Substituting the expression for \( D \) into the equation for \( C \), we get:

$$ C = 6 \cdot N \cdot B \cdot T \cdot S $$

Solving for \( S \):

$$ S = \frac{C}{6 \cdot N \cdot B \cdot T} $$

Assuming fixed context length of 32 (block size = 32) and batch size = 64, S can be simplified as:
$$ S = \frac{C}{12288 \cdot N} $$

In [10]:
import pandas as pd
# Create a DataFrame to summarize the model configurations and number of parameters
data = []
for model in GPT_model:
    config = GPT_configs[model]
    data.append({
        'model': model,
        'n_params': GPT_model[model].get_num_params(),

    })
df = pd.DataFrame(data)

sequence_length = 32
batch_size = 64
C_0 = 6e13
C_1 = 3e14
C_2 = 6e14
df['S_0'] = round(C_0 / (6 * df['n_params'] * sequence_length * batch_size))
df['S_1'] = round(C_1 / (6 * df['n_params'] * sequence_length * batch_size))
df['S_2'] = round(C_2 / (6 * df['n_params'] * sequence_length * batch_size))
df

,model,n_params,S_0,S_1,S_2
0,gpt-0,232992,20957.0,104785.0,209570.0
1,gpt-1,456672,10692.0,53461.0,106922.0
2,gpt-2,1015872,4807.0,24033.0,48065.0
3,gpt-3,1995136,2447.0,12237.0,24474.0
4,gpt-4,3977856,1227.0,6137.0,12275.0


Training steps table:
| model  | n_params | $S(Model, C_0)$    | $S(Model, C_1)$     | $ S(Model, C_2)$     |
|--------|----------|--------|---------|---------|
| gpt-0  | 232992   | 20957| 104785| 209570|
| gpt-1  | 456672   | 10692| 53461 | 106922|
| gpt-2  | 1015872  | 4807 | 24033 | 48065 |
| gpt-3  | 1995136  | 2447 | 12237 | 24474 |
| gpt-4  | 3977856  | 1227 | 6137  | 12275 |


In the case of the given exercise, it was used the laptop cpu for training all the models.

Command to train model:
```
python train.py config_train.py
```

For this exercise, each config file was created for each computational cost set, and GPT architecture. Using the same structure bellow:

In [ ]:
out_dir = 'out-gutenberg'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

dataset = 'gutenberg'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 32 # context of up to 256 previous characters

# Added to config file, based on GPT model
n_layer = 2
n_head = 2
n_embd = 96
dropout = 0.0

learning_rate = 1e-3
max_iters = 200 # Calculated using the equation deduced from the scaling laws
lr_decay_iters = max_iters # make equal to max_iters usually
min_lr = 1e-4
beta2 = 0.99

warmup_iters = 100

# on cpu, specify the device and compilation
device = 'cpu'  # run on cpu only
compile = False # do not torch compile the model

The results of each model are going to be saved and processed after for task 3, which requires the loss values from each training curve.

Task 3: Write down the final training loss values $L$ for each run. Plot $L$ as a function of $N$ and fit a curve for each compute value, see Figure 3 (left) Hoffmann et al. 2022. Extract the coordiantes of the minimum for each fitted curve. With the minimum values you can produce Figure 3 (center) and Figure 3 (right) of Hoffmann et al. 2022. Note, $N_{opt}\; = N_0 \; C^a$ and $D_{opt}\; = D_0\; C^b$. Write down the values for your your scaling laws and create a plot.


Anotações:

gpt0 - c0: loss = 1.7555

gpt1 - c0: loss = 1.6587

gpt2 - c0: loss = 1.6982

gpt3 - c0: loss =  1.7965

Anotações:

gpt1 - c1: loss = 1.4959

gpt2 - c1: loss =

gpt3 - c1: loss =